# yass! tour

In [2]:
# !rm -rf vampnet

In [3]:
%load_ext autoreload
%autoreload 2

## installing yass!

In [4]:
%%capture
!git clone --recursive https://github.com/hugofloresgarcia/vampnet.git

In [5]:
import os
os.chdir("./vampnet")
!pwd

/content/yass


In [13]:
%%capture
!pip install .

## quick start

set up our models, and put them in an interface

In [1]:
import audiotools as at
import vampnet

# load the audio tokenizer
codec = vampnet.load_codec()

# load the default pretrained model
model = vampnet.load_model("hugggof/vampnet:vampnet-base-best")

# put them into an interface
interface = vampnet.interface.Interface(codec, model)

/home/hugo/conda/envs/yass/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


loading model hugggof/yass:vampnet-base-best from the huggingface hub.


load some example audio

In [ ]:
# load an example audio file
signal = at.AudioSignal.excerpt("assets/example_audio/fountain.mp3", duration=5.0)

signal.widget()

/home/hugo/conda/envs/yass/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


loading model hugggof/yass:vampnet-base-best from the huggingface hub.
will process 1 chunks of length 430


vamping chunks: 1it [00:00,  1.74it/s]


encode the audio into acoustic tokens

In [ ]:
# get the tokens for the audio
tokens = interface.encode(signal)

# build a mask for the tokens
mask = interface.build_mask(tokens,
    periodic_prompt=3,
    upper_codebook_mask=1,
)

tokens.shape

generate output tokens with vampnet

In [ ]:
# generate the output tokens
output_tokens = interface.vamp(
    tokens, mask, return_mask=False,
    sampling_temperature=1.0,
    typical_filtering=True,
    top_p=0.8,
    sample_cutoff=1.0,
)

# convert them to a signal
output_signal = interface.to_signal(output_tokens).cpu()

# save the output signal
output_signal.write("scratch/output.wav")

## adding a dataset for fine tuning or training

yass uses a duckdb database to manage data.

first, let's init our db if we havent.

In [ ]:
vampnet.db.init()

running create_dataset_table
running create_audio_file_table
running create_ctrl_sig_table
running create_split_table
done! :)


now, let's add an audio folder to our database.

In [3]:
# this is what we will call our dataset
dataset = "example"

# create the dataset
vampnet.db.create.create_dataset(
    audio_folder="assets/example_audio",
    dataset_name=dataset,
)

Exception in WAL playback: Violates foreign key constraint because key "id: 1" does not exist in the referenced table



looking for audio files in assets/example_audio
Found 2 audio files


100%|██████████| 2/2 [00:00<00:00, 16.59it/s]

Processed 2 audio files
of which 0 failed
done! committing to the db.


preprocess the dataset

In [4]:
vampnet.db.preprocess.preprocess(dataset=dataset)

Found dataset example at assets/example_audio
Found 2 audio files
processing control signal dac


100%|██████████| 2/2 [00:03<00:00,  1.77s/it]

Processed 2 audio files
of which 0 failed
committing changes to the db.


pick a model to fine tune

In [5]:
# we'll just pick the first one available online.
available_hub_models = vampnet.list_hub_models()
print(f"available hub models: {available_hub_models}")

# load the hub model
model_id = available_hub_models[0]
print(f"using model: {model_id}")

available hub models: ['hugggof/yass:vampnet-base-best']
using model: hugggof/yass:vampnet-base-best


fine tune!

`yass.fine_tune.fine_tune` will return a `yass.Interface`, which we can use to upload our model to the HF hub.

In [6]:
finetuned_interface = vampnet.fine_tune.fine_tune(
    dataset=dataset,
    model_name=model_id,
    num_iters=1000, # fine tune for 1000 steps
)

seed: 10110100
fine-tuning, will validate on the training data!!!
Using 2 rows for split None
Using 2 rows for split None
Using 2 rows for split None
FINE TUNING!
loading model from hugggof/yass:vampnet-base-best
loading model hugggof/yass:vampnet-base-best from the huggingface hub.
Compiling model


tracker opened

Output()

Finished compiling model
initialized state.
trimmed batch size: 26
trimmed val batch size: 26
initialized dataloader.
marked only lora as trainable.
starting training loop.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 finetuned_interface = yass.fine_tune.fine_tune(                                              │
│   2 │   dataset=dataset,                                                                         │
│   3 │   model_name=model_id,                                                                     │
│   4 │   num_iters=1000, # fine tune for 1000 steps                                               │
│                                                                                                  │
│ /home/hugo/yass/yass/fine_tune.py:13 in fine_tune                                                │
│                                                                                                  │
│   10 │   with at.ml.Accelerator(amp=yass.AMP) as accel:                                          │
│   11 │   │   if accel.local_rank != 0:                                                           │
│   12 │   │   │   sys.tracebacklimit = 0                                                          │
│ ❱ 13 │   │   return train(accel,                                                                 │
│   14 │   │   │   save_path=yass.RUNS_DIR / f"finetuned-{dataset}",                               │
│   15 │   │   │   dataset=dataset,                                                                │
│   16 │   │   │   model_name=model_name,                                                          │
│                                                                                                  │
│ /home/hugo/conda/envs/yass/lib/python3.9/site-packages/torch/distributed/elastic/multiprocessing │
│ /errors/__init__.py:346 in wrapper                                                               │
│                                                                                                  │
│   343 │   │   │   assert error_handler is not None  # assertion for mypy type checker            │
│   344 │   │   │   error_handler.initialize()                                                     │
│   345 │   │   │   try:                                                                           │
│ ❱ 346 │   │   │   │   return f(*args, **kwargs)                                                  │
│   347 │   │   │   except ChildFailedError as e:                                                  │
│   348 │   │   │   │   rank, failure = e.get_first_failure()                                      │
│   349 │   │   │   │   if failure.error_file != _NOT_AVAILABLE:                                   │
│                                                                                                  │
│ /home/hugo/yass/yass/train.py:628 in train                                                       │
│                                                                                                  │
│   625 │   │                                                                                      │
│   626 │   │   while not done:                                                                    │
│   627 │   │   │   for tracker.step, batch in enumerate(train_dataloader, start=tracker.step):    │
│ ❱ 628 │   │   │   │   train_loop(state, batch, accel)                                            │
│   629 │   │   │   │                                                                              │
│   630 │   │   │   │   last_iter = (                                                              │
│   631 │   │   │   │   │   tracker.step == num_iters - 1 if num_iters is not None else False      │
│                                                                                                  │
│ /home/hugo/conda/envs/yass/lib/python3.9/site-packages/audi

## fine tuning a pretrained model

## pretrained models

view a list of locally available pretrained models, and load a local pretrained model

In [7]:
available_local_models = vampnet.list_local_models()
print(f"available_local_models: {available_local_models}")

# load the local model
model_name = available_local_models[0]
model = vampnet.load_local_model(model_name)

available_local_models: ['vampnet-base-best']


view a list of available pretrained models in the HF hub, and load one

In [ ]:
available_hub_models = vampnet.list_hub_models()
print(f"available hub models: {available_hub_models}")

# load the hub model
model_id = available_hub_models[0]
model = vampnet.load_hub_model(model_id)

available hub models: ['hugggof/yass:vampnet-base-best']


models/vampnet-base-best.vampnet:   0%|          | 0.00/12.0 [00:00<?, ?B/s]

### export
export the model after training.

this copies the best model from the `runs` directory to the `models` directory.
optionally, you can upload the model to the huggingface_hub.

this step is also necessary to use the model's gradio UI.

if you also want to upload to huggingface, make sure to run `huggingface-cli login` on your terminal first.


In [2]:
import yapecs
import vampnet
vampnet = yapecs.compose('vampnet', ['config/vampnet.py'])
vampnet.export.export_model()

Model saved locally to /home/hugo/yass/models/vampnet-base-best.vampnet
Creating repo hugggof/yass on the hub.
Uploading model to hugggof/yass


vampnet-base-best.vampnet:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

Model uploaded to hugggof/yass. !!!
you may now use the model in the interface by running `yass.load_hf_model('hugggof/yass:vampnet-base-best')`
